# Import necessary libraries

In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os, sys

from multiprocessing import Pool
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import *

from sklearn.ensemble import *
from sklearn.tree import *
from sklearn.naive_bayes import *
from sklearn.linear_model import *
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score

os.chdir("/home/sieu/PycharmProjects/ML-Vincent-Ng/project")

# Training import

In [2]:
column_name = pd.read_table('attr.txt', sep=":", usecols=all, names = ['attr', 'range'])
df = pd.read_table('train.txt', sep="\s+", usecols=all, names = list(column_name['attr']))

In [3]:
df = df.drop_duplicates()
df = df.dropna()
# Save year
X_year =  df.loc[:,'YEAR']
df.drop(['YEAR'], axis=1)

,B1,B2,B3,C1,C2,C3,C4,C5,C6,C7,...,CT22,CT23,CT24,CT25,CT26,CH1,CH2,CH3,CH4,Class
0,0,0,1,1.260,1.17,0.720,4.590,0.45,0.765,0.540,...,3,2,2,1,3,0,0,0,0,1
1,0,1,0,0.450,0.81,0.000,0.000,0.00,0.855,0.000,...,3,2,1,1,3,0,0,0,0,1
2,0,0,1,0.540,2.88,0.000,0.000,0.00,0.765,0.000,...,3,2,2,1,3,0,0,0,0,1
3,0,0,1,0.810,1.35,0.450,0.000,0.00,0.000,0.720,...,3,2,2,1,3,0,0,0,0,1
4,0,0,1,0.900,1.17,0.765,0.000,0.00,0.630,0.810,...,4,3,2,1,3,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0,1,0,0.450,1.17,0.630,1.215,0.45,0.990,0.540,...,3,2,2,1,3,0,0,0,0,0
11996,0,1,0,5.085,0.90,0.000,0.000,0.00,0.000,0.945,...,4,3,2,1,3,0,0,0,0,1
11997,0,0,1,1.170,1.71,0.630,0.000,0.00,0.720,0.000,...,3,2,1,1,3,0,0,0,0,0
11998,0,0,1,0.810,1.17,0.000,0.000,0.00,0.990,0.810,...,5,3,2,2,4,0,0,0,0,1


In [4]:
# Seperate data by type
y = df["Class"].to_numpy()
X_B = df.loc[:, 'B1':'B3']
X_C = df.loc[:,'C1':'C139']
X_CT = df.loc[:,'CT1':'CT26']
X_H = df.loc[:,'CH1':'CH4']

In [ ]:
# Calculate statistics
print(X_B.describe())
print(X_C.describe())
print(X_CT.describe())
print(X_H.describe())
print(X_year.describe())
print(np.unique(df[["CT22"]]))

# Select good features

In [ ]:
# Below are the columns retained by each
# List contains all attributes retained during feature selection: attr_lst
x_c_cols = X_C.columns
x_ct_cols = X_CT.columns
x_h_cols = X_H.columns
# Feature selection based on variance threshold for every type of data
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))

# X_C dataframe
sel.feature_names_in_= x_c_cols
X_C = sel.fit_transform(X_C)
x_c_cols=sel.get_feature_names_out(x_c_cols)
attr_lst=x_c_cols
print(x_c_cols)
print("Number of C* features: ", len(x_c_cols))

# X_CT dataframe
sel.feature_names_in_= x_ct_cols
X_CT = sel.fit_transform(X_CT)
x_ct_cols=sel.get_feature_names_out(x_ct_cols)
attr_lst=np.concatenate((attr_lst, x_ct_cols))
print(x_ct_cols)
print("Number of CT* features: ", len(x_ct_cols))

# X_H dataframe
sel.feature_names_in_= x_h_cols
X_H = sel.fit_transform(X_H)
x_h_cols=sel.get_feature_names_out(x_h_cols)
attr_lst=np.concatenate((attr_lst, x_h_cols))
print(x_h_cols)
print("Number of CH* features: ", len(x_h_cols))

In [ ]:
selection_factor = 5
attr_lst = []

if len(x_c_cols) > 0:    
    print("BEGIN: Ridge classifier feature selection for C* atttributes")
    sel_X_C = SequentialFeatureSelector(estimator=RidgeClassifierCV(), n_features_to_select=max(len(x_c_cols)//(selection_factor), 1), direction="forward")
    sel_X_C.feature_names_in_= x_c_cols
    X_C = sel_X_C.fit_transform(X_C, y)
    x_c_cols = sel_X_C.get_feature_names_out(x_c_cols)
    attr_lst=np.concatenate((attr_lst, x_c_cols))
    print(x_c_cols)
    print("Done Ridge classifier feature selection for C* atttributes")

if len(x_ct_cols) > 0:
    print("BEGIN: CategoricalNB classifier feature selection for CT* atttributes")
    sel_X_CT = SequentialFeatureSelector(estimator=CategoricalNB(), n_features_to_select=max(len(x_ct_cols)//selection_factor, 1), direction="forward")
    sel_X_CT.feature_names_in_= x_ct_cols
    X_CT = sel_X_CT.fit_transform(X_CT, y)
    x_ct_cols = sel_X_CT.get_feature_names_out(x_ct_cols)
    attr_lst=np.concatenate((attr_lst, x_ct_cols))
    print(x_ct_cols)
    print("Done CategoricalNB classifier feature selection for CT* atttributes")

if len(x_h_cols) > 0:
    print("BEGIN: DecisionTreeClassifier feature selection for CH* atttributes")
    sel_X_H= SequentialFeatureSelector(estimator=DecisionTreeClassifier(), n_features_to_select=max(len(x_h_cols)//selection_factor, 1), direction="forward")
    sel_X_H.feature_names_in_= x_h_cols
    X_H = sel_X_H.fit_transform(X_H, y)
    x_h_cols = sel_X_H.get_feature_names_out(x_h_cols)
    attr_lst=np.concatenate((attr_lst, x_h_cols))
    print(x_h_cols)
    print("Done DecisionTreeClassifier feature selection for CH* atttributes")

X=np.concatenate((X_C, X_CT, X_H), axis=1)
print(attr_lst)
print(X.shape)

In [ ]:
# Save feature selection results
with open("_fet.pkl", 'wb') as file:
    pickle.dump(attr_lst, file)

# Load good feature

In [48]:
# Load good feature list
with open("25_fet.pkl", 'rb') as file:
    attr_lst = pickle.load(file)
print(attr_lst)
print(len(attr_lst))

['C1' 'C3' 'C5' 'C6' 'C8' 'C13' 'C16' 'C17' 'C19' 'C26' 'C31' 'C48' 'C53'
 'C59' 'C64' 'C69' 'C70' 'C94' 'C97' 'CT3' 'CT16' 'CT26' 'CH3']
23


In [5]:
# Load from clipboard
attr_lst = ['C1', 'C3', 'C5', 'C6', 'C8', 'C13', 'C16', 'C17', 'C19', 'C26', 'C31', 'C48', 'C53', 'C59', 'C64', 'C69', 'C70', 'C94', 'C97', 'CT3', 'CT16', 'CT26', 'CH3']
attr_lst=np.array(attr_lst)

# PCA

In [7]:
attr_lst=attr_lst.astype(str)
print(attr_lst)

['C1' 'C3' 'C5' 'C6' 'C8' 'C13' 'C16' 'C17' 'C19' 'C26' 'C31' 'C48' 'C53'
 'C59' 'C64' 'C69' 'C70' 'C94' 'C97' 'CT3' 'CT16' 'CT26' 'CH3']


In [57]:
# Naive PCA
X = df[attr_lst].to_numpy()
pca = PCA(svd_solver='full', n_components='mle')
X = pca.fit_transform(X)
X_new=np.concatenate((X_B, X), axis=1)
X_new.shape

(11982, 25)

In [6]:
# Reseperate C data to perform PCA
non_C_ind = np.logical_or((np.core.defchararray.find(attr_lst,"CT")!=-1),(np.core.defchararray.find(attr_lst,"CH")!=-1))
C_label = attr_lst[np.logical_not(non_C_ind)]
attr_lst = attr_lst[non_C_ind]
print(C_label)
X = df[C_label].to_numpy()
print('Perform PCA on continuous features to filter noises')
pca = PCA(svd_solver='full', n_components='mle')
X = pca.fit_transform(X)
no_component = len(pca.get_feature_names_out())
print(no_component)
# Concat X_B with X to get the final attribute array
X_new=np.concatenate((X_B, df[attr_lst], X), axis=1)
X_new.shape

['C1' 'C3' 'C5' 'C6' 'C8' 'C13' 'C16' 'C17' 'C19' 'C26' 'C31' 'C48' 'C53'
 'C59' 'C64' 'C69' 'C70' 'C94' 'C97']
Perform PCA on continuous features to filter noises
18


(11982, 25)

# GridSearch for best model

In [ ]:
# Naive PCA
params = {'max_leaf_nodes':np.arange(200,500, 20), 'l2_regularization':np.arange(0.5,1,0.05), 'learning_rate':np.arange(0.3,1,0.05), 'max_depth':np.arange(8, 40, 2), 'max_bins':np.arange(70, 150, 2)}
#params = {'max_iter':np.arange(50,200,100),'max_leaf_nodes':np.arange(100,500, 10), 'l2_regularization':np.arange(0,1,0.05)}
search = GridSearchCV(estimator=HistGradientBoostingClassifier(categorical_features=[0, 1, 2]), 
                                param_grid=params).fit(X_new,y)

In [ ]:
# Normal Grid
params = {'max_leaf_nodes':np.arange(100,250, 10), 'l2_regularization':np.arange(0.5,1,0.05), 'learning_rate':np.arange(0.05,0.3,0.05), 'max_depth':np.arange(6, 18, 1), 'max_bins':np.arange(20, 70, 2)}
#params = {'max_iter':np.arange(50,200,100),'max_leaf_nodes':np.arange(100,500, 10), 'l2_regularization':np.arange(0,1,0.05)}
search = GridSearchCV(estimator=HistGradientBoostingClassifier(max_iter=1000, categorical_features=np.arange(0,X_new.shape[1]-no_component), early_stopping=True), 
                                param_grid=params, n_jobs=-1).fit(X_new,y)

In [79]:
# Best model validation
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle=True)
arr = []
for train_index , test_index in kfold.split(X):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2, 4,5,6], max_leaf_nodes=300,max_iter=300,learning_rate=0.5,l2_regularization=0.95).fit(X_train, y_train)
    #print(clf.score(X_test, y_test))
    #arr.append(clf.score(X_test, y_test))
    arr.append(search.score(X_test, y_test))
    print(search.score(X_test, y_test))
print("Done")
print("mean acc: ", np.mean(arr))

0.8982485404503753
0.8840700583819849
0.8898163606010017
0.9048414023372288
0.8973288814691152
0.8898163606010017
0.9106844741235393
0.9098497495826378
0.9090150250417363
0.9031719532554258
Done
mean acc:  0.8996842805844046


In [13]:
# Save models
with open("927_newmethod.pkl", 'wb') as file:
    pickle.dump(search, file)

In [ ]:
# Load model
with open("888_low_depth_leaf_model_newmethod.pkl", 'rb') as file:
    search = pickle.load(file)

In [80]:
search.best_params_

{'max_leaf_nodes': 200,
 'max_depth': 36,
 'max_bins': 118,
 'learning_rate': 0.3,
 'l2_regularization': 0.55}

# Essemble

In [ ]:
def Ridge_classifiers(data):
    clf = RidgeClassifier(solver="svd")
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def knn_classifiers(data):
    clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def svc_classifiers(data):
    clf = SVC(kernel='rbf', degree=3)
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def BernoulliNB_classifiers(data):
    clf = BernoulliNB()
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)
def HGB_classifiers(data):
    clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2])
    clf.fit(data[0], data[1])
    score = clf.score(data[2], data[3])
    return (clf, score)

In [ ]:
kfold = sklearn.model_selection.KFold(n_splits=10, shuffle=True)
dataset = []
for train_index , test_index in kfold.split(X):
    X_train, X_test = X_new[train_index], X_new[test_index]
    y_train, y_test = y[train_index], y[test_index]
    dataset.append((X_train, y_train, X_test, y_test))
    #Testing individual clf
    clf = HistGradientBoostingClassifier(categorical_features=[0, 1, 2], max_leaf_nodes=800,max_iter=300,learning_rate=0.8500000000000002,l2_regularization=0.8500000000000001).fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
    #clf = RidgeClassifier(solver="svd").fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
    #clf = BernoulliNB().fit(X_train, y_train)
    #assembly.append(clf)
    #vote_weights.append(clf.score(X_test, y_test))
print("Done")

In [ ]:
assembly = []
vote_weights = []
def append_results(result):
    for clf, score in result:
        assembly.append(clf)
        vote_weights.append(score)
if __name__ == '__main__':
    p = Pool()
    p.map_async(Ridge_classifiers, dataset, callback=append_results)
    #p.map_async(nn_classifiers, dataset, callback=append_results)
    #p.map_async(knn_classifiers, dataset, callback=append_results)
    #p.map_async(svc_classifiers, dataset, callback=append_results)
    p.map_async(BernoulliNB_classifiers, dataset, callback=append_results)
    p.map_async(HGB_classifiers, dataset, callback=append_results)
    p.close()
    p.join()

In [ ]:
print(assembly)
print(vote_weights)

In [ ]:
clfs = []
for i in range(len(assembly)):
    clfs.append(("Classifier " + str(i), assembly[i]))
eclf = VotingClassifier(estimators=clfs,voting='hard', weights=vote_weights, n_jobs=-1)

In [ ]:
cross_val_score(eclf, X_new, y, scoring='accuracy', cv=10, n_jobs=-1)

# Evaluation 

In [ ]:
attr = pd.read_table('attr.txt', sep=":", usecols=all, names = ['attr', 'range'])
dfX_pred = pd.read_table('sample-test.txt', sep="\s+", usecols=all, names = list(attr['attr'])[:-1])
dfX_pred = dfX_pred.dropna()

In [ ]:
# Seperate prediction data by type
X_pred_B = dfX_pred.loc[:, 'B1':'B3']
X_pred_C = dfX_pred.loc[:,'C1':'C139']
X_pred_CT = dfX_pred.loc[:,'CT1':'CT26']
X_pred_H = dfX_pred.loc[:,'CH1':'CH4']

In [ ]:
with open("25_fet.pkl", 'rb') as file:
    attr_lst = pickle.load(file)
list(attr_lst)

In [ ]:
# Naive PCA
X_pred = dfX_pred[attr_lst].to_numpy()
pca = PCA(svd_solver='full', n_components='mle')
X_pred = pca.fit_transform(X_pred)
X_pred_new=np.concatenate((X_pred_B, X_pred), axis=1)
X_pred_new.shape

In [ ]:
# Continuous PCA
# Reseperate C data to perform PCA
non_C_ind = np.logical_or((np.core.defchararray.find(attr_lst,"CT")!=-1),(np.core.defchararray.find(attr_lst,"CH")!=-1))
C_label = attr_lst[np.logical_not(non_C_ind)]
print(C_label)
X_pred = dfX_pred[C_label].to_numpy()
print('Perform PCA on continuous features to filter noises')
pca = PCA(svd_solver='full', n_components=no_component)
X_pred = pca.fit_transform(X_pred)
# Concat X_B with X to get the final attribute array
X_pred_new=np.concatenate((X_pred_B, dfX_pred[attr_lst[non_C_ind]], X_pred), axis=1)
X_pred_new.shape

In [ ]:
# Load model then predict and save prediction
with open("91_model.pkl", 'rb') as file:
    search = pickle.load(file)
y_pred = search.predict(X_pred_new)
np.savetxt(fname="prediction.txt", X=y_pred, fmt='%d')